In [ ]:
'''
Questo script esegue il training della rete
sull'intero dataset
'''

In [ ]:
# Import delle librerie
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [ ]:
# Import funzioni da file utils.txt
import ipynb
from ipynb.fs.full.utils import fix_seed, train_one_epoch, show_batch

In [ ]:
IMGS_PATH = '/media/users/cgambina/Progetto_6/Dati/Immagini'

# Definizione iper-parametri migliori
NUM_EPOCHS = 9
BS = 128
LR = 1e-3
LR_DEC = 0.75

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Inizializzazione generatore valori random
SEED = 123
fix_seed(SEED)

In [ ]:
# Definizione trasformazioni da applicare al batch di immagini
transforms = T.Compose([
    T.RandomChoice(
        [
            T.RandomRotation((-5,5)),
            T.RandomRotation((85,95)),
            T.RandomRotation((175,195)),
            T.RandomRotation((265,275)),
    ]),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
dataset = ImageFolder(IMGS_PATH, transforms)
criterion = nn.BCEWithLogitsLoss() 

net = resnet50(weights='ResNet50_Weights.IMAGENET1K_V1').to(device)
n_filters = net.fc.in_features
net.fc = nn.Linear(n_filters, 1).to(device) 

In [ ]:
# Riduzione del learning rate in base al layer (blocco)
layer_names = []
for name, param in net.named_parameters():
    layer_names.append(name)

layer_names.reverse()

parameters      = []
prev_group_name = layer_names[0].split('.')[0]

for idx, name in enumerate(layer_names):
    
    cur_group_name = name.split('.')[0]
    if cur_group_name != prev_group_name:
        LR *= LR_DEC
    prev_group_name = cur_group_name

    #print(f'{idx}: lr = {LR:.6f}, {name}')
    parameters += [{'params': [p for n, p in net.named_parameters() if n == name and p.requires_grad],
                    'lr':     LR}]

In [ ]:
optimizer = optim.Adam(parameters)  

In [ ]:
# Training della rete
train_loader = DataLoader(dataset, batch_size=BS)

for epoch in range(NUM_EPOCHS):
    train_loss, train_correct = train_one_epoch(net,device,train_loader,criterion,optimizer,scheduler=None)
    train_loss = train_loss / len(train_loader.sampler)
    train_acc = train_correct / len(train_loader.sampler) * 100
    print(f'Epoca:{epoch+1}/{NUM_EPOCHS} Training Loss:{train_loss:.3f} Training Acc {train_acc:.2f} %') 

In [ ]:
# Salvataggio dei paramatri (pesi) della rete
save_dir = '/media/users/cgambina/Progetto_6/Dati/BestWeights'
save_path = f'{save_dir}/best_model.pt'

torch.save(net.state_dict(), save_path)

In [ ]:
'''
# Mostra un campione di immagini
show_tfms = T.Compose([
    T.RandomChoice(
        [
            T.RandomRotation((-5,5)),
            T.RandomRotation((85,95)),
            T.RandomRotation((175,195)),
            T.RandomRotation((265,275)),
    ]),
    T.ToTensor(),
])

foo_dataset = ImageFolder(IMGS_PATH, show_tfms)
show_batch(foo_dataset)
'''